In [11]:
for i in range(0, 6, 2):
    print(i)
    print(i+1)
print(i+2)

0
1
2
3
4
5
6


In [2]:
import torch
def outer_product(x, y):
    """
    x: [batch_size, x_dim]
    y: [batch_size, y_dim]
    """
    x = x.unsqueeze(2)
    y = y.unsqueeze(1)
    out = x * y
    out = out.reshape(x.size(0), -1)
    return out
x = torch.randn(3, 95)
y = torch.randn(3, 5)
outer_product(x, y).shape

torch.Size([3, 95, 1])
torch.Size([3, 1, 5])


torch.Size([3, 95, 5])

In [5]:
import torch
torch.__version__

'2.0.0'

In [8]:
# caculate the improvement percentage
import numpy as np
def cac_improvement(old, new):
    return (new - old) / old * 100

new = np.array([0.6802,0.8008,0.8140,0.8136,0.8057,0.8292])
old = np.array([0.6711,0.7865,0.8032,0.7889,0.7685,0.8211])
new = np.array([0.0321,0.1037,0.0939,0.0976,0.1401, 0.1621])
old = np.array([0.0330,0.1116,0.0951,0.0997,0.1425,0.1651])
cac_improvement(old, new).mean()
    

-2.7792609112148328

In [3]:
import torch
import torch.nn as nn
x  = torch.randn(3, 1)
layer = nn.Linear(1, 5)
y = layer(x)
print(x)
print(y)

tensor([[ 1.3127],
        [-1.6860],
        [-0.7457]])
tensor([[ 0.3166,  0.2711, -0.7014,  0.2126,  0.2096],
        [ 1.7651,  1.0390,  0.8166,  1.9816,  0.0379],
        [ 1.3109,  0.7982,  0.3406,  1.4269,  0.0918]],
       grad_fn=<AddmmBackward0>)


In [126]:

import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Load the dataset from online 
url = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv'
# url = './data_sample/diabetes.csv'
data = pd.read_csv(url, header=None)
# oversample the negative samples
sample_ratio = 0.5
negative_sample_num = round(len(data[data[8] == 0]) * sample_ratio)
data = pd.concat([data[data[8] == 1], data[data[8] == 0].sample(negative_sample_num, replace=True)])
# Separate the features and target variable
label = 8
X = data.drop(label, axis=1)
y = data[label]
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=46)

# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Convert the data to PyTorch tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train.values, dtype=torch.float32)
y_test = torch.tensor(y_test.values, dtype=torch.float32)
class Netron(nn.Module):
    def __init__(self, input_dim):
        super(Netron, self).__init__()
        self.fc1 = nn.Linear(input_dim, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)
        x = self.sigmoid(x)
        return x
# Set the random seed for reproducibility
torch.manual_seed(4)

# Initialize the model
model = Netron(input_dim=X_train.shape[1])

# Define the loss function and optimizer
criterion = nn.BCELoss(reduction='none')
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the model
num_epochs = 10
batch_size = 32
num_batches = len(X_train) // batch_size

for epoch in range(num_epochs):
    for batch in range(num_batches):
        # Get the batch data
        start_idx = batch * batch_size
        end_idx = (batch + 1) * batch_size
        batch_X = X_train[start_idx:end_idx]
        batch_y = y_train[start_idx:end_idx]

        # Forward pass
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y.unsqueeze(1))
        # caculate the loss of positive samples
        loss = loss*y_train + (1-y_train)*loss
        loss = loss.sum()

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # Print the loss for each epoch
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')



# Evaluate the model on the test set
with torch.no_grad():
    model.eval()
    y_pred = model(X_test)
    y_pred = sample_ratio*y_pred/(1-(1-sample_ratio)*y_pred)
    # y_pred = (y_pred >= 0.5).float()

    # accuracy = (y_pred == y_test.unsqueeze(1)).sum().item() / len(y_test)
    # print(f'Test Accuracy: {accuracy:.4f}')
    # caculate the mean of y_pred[y=1]
    print(f'prediction mean: {y_pred[y_test==1].mean():.4f}')
    print(f'positive sample: {len(y_test[y_test==1])}')
    print(f'negative sample: {len(y_test[y_test==0])}')
    print(f'actual mean: {y_test.mean():.4f}')


Epoch [1/10], Loss: 7660.5166
Epoch [2/10], Loss: 6921.8589
Epoch [3/10], Loss: 6487.2461
Epoch [4/10], Loss: 6202.7012
Epoch [5/10], Loss: 6017.5674
Epoch [6/10], Loss: 5913.8203
Epoch [7/10], Loss: 5861.7832
Epoch [8/10], Loss: 5835.8735
Epoch [9/10], Loss: 5821.7402
Epoch [10/10], Loss: 5813.2549
prediction mean: 0.5115
positive sample: 51
negative sample: 53
actual mean: 0.4904


In [75]:
len(y_pred[y_test==1])

59

In [16]:
import tensorflow as tf
# create a tensorflow layer with dimension 48-1
layer = tf.keras.layers.Dense(1, input_shape=(48,))
# create a input of (48,48)
input1 = tf.random.normal([100, 48])
input2 = tf.random.normal([100, 48])
input3= tf.random.normal([100, 48])
input4= tf.random.normal([100, 48])
experts = [input1, input2, input3]
input = tf.concat(experts, axis=1)
gate = tf.random.normal([100, 27])
gate_out_reshape = tf.reshape(gate, [-1, 9, 3])
gate_softmax = tf.nn.softmax(gate_out_reshape)
mmoe_mlp_result_input = tf.keras.layers.Concatenate(axis=1)([tf.expand_dims(v,1) for v in experts])
mmoe_mlp_result = tf.matmul(gate_softmax, mmoe_mlp_result_input)
mlp_shape = mmoe_mlp_result.shape
mmoe_mlp_splits = [tf.squeeze(v, 1) for v in tf.split(mmoe_mlp_result, mlp_shape[1], axis=1)]

# print the output
print(mmoe_mlp_splits[0])
#
print(tf.keras.backend.eval(gate_softmax))
print(len(mmoe_mlp_splits))




Tensor("Squeeze_72:0", shape=(100, 48), dtype=float32)
[[[0.10409561 0.6774731  0.21843123]
  [0.32537004 0.2582363  0.4163937 ]
  [0.25499988 0.14607085 0.5989293 ]
  ...
  [0.25034818 0.17410088 0.575551  ]
  [0.28291398 0.11199827 0.6050877 ]
  [0.28036767 0.37287846 0.34675387]]

 [[0.8757716  0.00406646 0.12016194]
  [0.14758863 0.23786373 0.6145476 ]
  [0.43321624 0.2891689  0.27761486]
  ...
  [0.09620936 0.5757924  0.32799825]
  [0.41422153 0.45843616 0.12734237]
  [0.53675884 0.17624821 0.2869929 ]]

 [[0.14462696 0.08422034 0.7711527 ]
  [0.13770004 0.5475965  0.3147035 ]
  [0.11975987 0.54444075 0.3357994 ]
  ...
  [0.5229638  0.16491988 0.31211627]
  [0.06484465 0.48299566 0.4521597 ]
  [0.68803877 0.08180112 0.23016007]]

 ...

 [[0.11332358 0.35908467 0.52759176]
  [0.17994204 0.06786871 0.7521893 ]
  [0.48757085 0.3269805  0.18544859]
  ...
  [0.0825779  0.69366515 0.223757  ]
  [0.25346154 0.5619587  0.18457976]
  [0.15865892 0.1133096  0.72803146]]

 [[0.0663952  0.911

In [4]:
# ----------- Embedding
cate_input = [["music", "movie", "finance"],["game", "military", "history"]]  # [batch_size,1]的string型"文章分类"向量
cate_ids = id_mapping_layer(cate_input)  # string型输入的“文章分类”映射成int型id
# 得到形状=[batch_size,4]的float稠密向量，表示每个“文章分类”的语义
cate_embeddings = emb_layer(cate_ids)

NameError: name 'id_mapping_layer' is not defined

In [ ]:
import numpy as np
import tensorflow as tf

y_label = np.array([10, 4, 7, 2, 21, 7, 50, 8, 8, 2, 2, 2, 2, 2, 2, 2]) * 0.01
y_pred = np.array([10, 4, 7, 2, 21, 7, 50, 8, 8, 2, 2, 2, 2, 2, 2, 2]) * 0.01 + 0.001
# convert numpy array to tensor
y_label = tf.convert_to_tensor(y_label, dtype=tf.float32)
y_pred = tf.convert_to_tensor(y_pred, dtype=tf.float32)
# stack y_label with 1-y_label
y_label_stack = tf.stack([y_label, 1 - y_label], axis=1)
y_pred_stack = tf.stack([y_pred, 1 - y_pred], axis=1)

# print(tf.keras.backend.eval(y_label))
# print(tf.keras.backend.eval(y_pred))
kl = tf.keras.losses.KLDivergence(reduction="sum")
kl_loss = kl(y_label, y_pred)
kl_loss_stack = kl(y_label_stack, y_pred_stack)
print(tf.keras.backend.eval(kl_loss))
print(tf.keras.backend.eval(kl_loss_stack))

# # print y_label's value
# print(y_label)

In [24]:
import numpy as np
import tensorflow as tf
y_pred = np.random.randn(10, 5)
y_label = np.random.randn(10, 5)
pctr = np.random.randn(10)
# convert numpy array to tensor
y_pred = tf.convert_to_tensor(y_pred, dtype=tf.float32)
y_label  = tf.convert_to_tensor(y_label, dtype=tf.float32)
pctr = tf.convert_to_tensor(pctr, dtype=tf.float32)
multitask_loss = tf.nn.sigmoid_cross_entropy_with_logits(labels=y_label, logits=y_pred)
mask = tf.cast(tf.not_equal(y_label, 0), tf.float32)
masked_loss = tf.multiply(multitask_loss, mask)
loss = tf.reduce_sum(masked_loss, axis=1)
loss


<tf.Tensor 'Sum_11:0' shape=(10,) dtype=float32>

: 

In [9]:
def func1(*args,**kwargs):
	print(type(args))
	print(args)
	print(type(kwargs))
	print(kwargs)
func1(1,2,'hello world',a=4,b=6,c='oo',d=[])


<class 'tuple'>
(1, 2, 'hello world')
<class 'dict'>
{'a': 4, 'b': 6, 'c': 'oo', 'd': []}


In [87]:
impory

,Method,fr/ctcvr_auc,fr/ctr_auc,fr/cvr_auc,0,1,2,method
0,DESCM_Embedding+Basic_Loss,0.8593,0.7144,0.7961,fr,Basic_Loss,DESCM_Embedding,DESCM_Embedding+Basic_Loss
1,ESCM+Basic_Loss,0.8438,0.7148,0.7782,fr,Basic_Loss,ESCM,ESCM+Basic_Loss
2,DESCM_Embedding+Entire_Space_Basic_Loss,0.8392,0.7140,0.7370,fr,Entire_Space_Basic_Loss,DESCM_Embedding,DESCM_Embedding+Entire_Space_Basic_Loss
3,DESCM_Embedding+DR_Loss,0.8610,0.6874,0.7919,fr,DR_Loss,DESCM_Embedding,DESCM_Embedding+DR_Loss
4,DESCM_Embedding+IPW_Loss,0.8634,0.6839,0.7920,fr,IPW_Loss,DESCM_Embedding,DESCM_Embedding+IPW_Loss
5,ESCM+Entire_Space_Basic_Loss,0.8172,0.7153,0.6881,fr,Entire_Space_Basic_Loss,ESCM,ESCM+Entire_Space_Basic_Loss
6,ESCM+DR_Loss,0.8687,0.6853,0.7927,fr,DR_Loss,ESCM,ESCM+DR_Loss
7,ESCM+IPW_Loss,0.8707,0.6817,0.7919,fr,IPW_Loss,ESCM,ESCM+IPW_Loss


In [2]:
s = 'dbdsa'
sorted(s)

['a', 'b', 'd', 'd', 's']

In [24]:
import torch
import torch.nn as nn
class CrossNetwork(nn.Module):
    def __init__(self, input_dim, layer_num=3):
        super(CrossNetwork, self).__init__()
        self.cross_layers = nn.ModuleList([nn.Linear(input_dim, 1) for _ in range(layer_num)])

    def forward(self, x, x0):
        for layer in self.cross_layers:
            x = x0 * layer(x) + x
        return x

In [5]:
import numpy as np
best = np.array([0.6789])
sub = np.array([0.6691])
def caculate_improvement(best, sub):
    return (best-sub)/sub*100
caculate_improvement(best, sub)



array([1.46465401])

In [21]:
import numpy as np
base = '''0.5582
0.6655
0.6665
0.6419
0.6780
0.6248
0.5378
0.6559
0.6642
0.6430
0.6723
0.6194
0.6206
0.7144
0.7158
0.6890
0.7129
0.6451
0.5621
0.6676
0.6671
0.6498
0.6817
0.5993'''
ichm ='''0.5654
0.6700
0.6724
0.6699
0.6241
0.6364
0.5605
0.6681
0.6842
0.6602
0.6811
0.6300
0.6216
0.7101
0.7113
0.6905
0.7142
0.6452
0.6091
0.7062
0.6947
0.6712
0.6807
0.6391'''


base = np.array([float(i) for i in base.split('\n')]).reshape(4,6)
ichm = np.array([float(i) for i in ichm.split('\n')]).reshape(4,6)

def get_improve(base, ichm):
    return (ichm-base)/base*100
improve = get_improve(base, ichm)
improve = np.mean(improve, axis=1)
print(base)
print(ichm)
print(improve)

# print(improve)

[[0.5582 0.6655 0.6665 0.6419 0.678  0.6248]
 [0.5378 0.6559 0.6642 0.643  0.6723 0.6194]
 [0.6206 0.7144 0.7158 0.689  0.7129 0.6451]
 [0.5621 0.6676 0.6671 0.6498 0.6817 0.5993]]
[[0.5654 0.67   0.6724 0.6699 0.6241 0.6364]
 [0.5605 0.6681 0.6842 0.6602 0.6811 0.63  ]
 [0.6216 0.7101 0.7113 0.6905 0.7142 0.6452]
 [0.6091 0.7062 0.6947 0.6712 0.6807 0.6391]]
[ 0.18667611  2.46455249 -0.10897908  4.6780713 ]
